In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import crm_agent
import pandas as pd
import glob
import os

In [10]:
all_tasks = ['case_routing', 'activity_priority', 'monthly_trend_analysis', 'top_issue_identification', 'lead_routing', 'named_entity_disambiguation', 'lead_qualification', 'quote_approval', 'knowledge_qa']

pre_text = {'content': 'You are a lead qualification assistant.\nYou will be given call transcript excerpts (tool output).\nReturn exactly ONE Action command of the form:\n<respond> LABELS </respond>\nwhere LABELS is either:\n- None\n- or a comma-separated subset of: Authority, Budget, Timeline, Need\nNo other text.',
   'role': 'system'}
post_text = {'content': 'Now provide the final qualification result.\nReturn exactly one action command:\n<respond> LABELS </respond>\nLABELS must be None or a comma-separated subset of: Authority, Budget, Timeline, Need.',
   'role': 'user'}

In [4]:
ts = crm_agent.capture_results('ollama/deepseek-v3.1:671b-cloud', 'lead_qualification', 'ollama', 10)

Task 10 completed.


In [34]:
ts[1][0]

[{'role': 'system',
  'content': "You are an expert in Salesforce and you have access to a Salesforce instance. \n\n# Instructions\n- You will be provided a question, the system description, and relevant task context.\n- Interact with the Salesforce instance to build Salesforce Object Query Language (SOQL) or Salesforce Object Search Language (SOSL) queries as appropriate, to help you answer the question.\n- Salesforce Object Search Language (SOSL) can be used to construct text-based search queries against the search index.\n- Your generation should always be an Action command and NOTHING ELSE. Generate only one Action command.\n- DO NOT generate ANY system observation, you will receive this based on your Action command.\n- If no record is found matching the requirements mentioned, just return 'None'.\n- If the user's request is unclear or under-specified, use the respond action to ask for clarification before proceeding with queries or submitting an answer.\n\nHere is a description of

In [11]:
def capture_and_clean(message_exchange: list):
    cleaned_messages = []
    for message in message_exchange:
            if message['role'] == 'system':
                message = pre_text
            cleaned_messages.append(message)
            if message['role'] == 'user' and 'Salesforce instance output:' in message['content']:
                cleaned_messages.append(post_text)
                break
    return cleaned_messages


In [27]:
test_messages = []
test_actual_answers = []
for test in ts:
    test_messages.append(capture_and_clean(test[0]))
    test_actual_answers.append(test[2]['answer'])


In [31]:
pd.DataFrame({
    'message': test_messages,
    'answer': test_actual_answers
}).to_json('Lead_Qualification_Qwen_test.jsonl', orient='records', lines=True)

In [31]:
ned_results = crm_agent.capture_results('ollama/deepseek-v3.1:671b-cloud', 'named_entity_disambiguation', 'ollama')

Task 10 completed.
Task 20 completed.
Task 30 completed.
Task 40 completed.
Task 50 completed.
Task 60 completed.
Task 70 completed.
Task 80 completed.
Task 90 completed.
Task 100 completed.


In [32]:
transformed_data = []
for cap in ned_results:
    messages = cap[0]
    executions = cap[1].get('executions', [])
    if executions:
        query = executions[0]
    else:
        query = {}
    transformed_pair = {
            "instruction": messages[0]['content'] if messages[0]['role'] == 'system' else "",
            "input": messages[1]['content'] if messages[1]['role'] == 'user' else "",
            "output": query.get('query', "")
        }
    transformed_data.append(transformed_pair)


In [33]:
transformed_df = pd.DataFrame(transformed_data)
# df_train = transformed_df.sample(frac=0.9, random_state=42)
# df_test = transformed_df.drop(transformed_df.index)

# transformed_df.to_json('lead_qualification_train.jsonl', orient='records', lines=True)
# transformed_df.to_json('activity_priority_train.jsonl', orient='records', lines=True)

In [34]:
transformed_df

,instruction,input,output
0,You are an expert in Salesforce and you have a...,Display the power optimization manager that I ...,
1,You are an expert in Salesforce and you have a...,Can you display the interactive engineering ap...,
2,You are an expert in Salesforce and you have a...,Can you display the AI-Driven Circuit Analyzer...,
3,You are an expert in Salesforce and you have a...,Display the cloud-based design program that I ...,
4,You are an expert in Salesforce and you have a...,Display the Core Cybersecurity Solution I purc...,
...,...,...,...
95,You are an expert in Salesforce and you have a...,Could you display the cybersecurity software p...,SELECT AccountId FROM Contact WHERE Id = '003W...
96,You are an expert in Salesforce and you have a...,Can you display the professional heartbeat sim...,
97,You are an expert in Salesforce and you have a...,Display the complete security management softw...,
98,You are an expert in Salesforce and you have a...,Display the simulation tool I purchased five d...,


In [52]:
# Messages
msg = cap_results[0][0]
msg[0]

{'role': 'system',
 'content': "You are an expert in Salesforce and you have access to a Salesforce instance. \n\n# Instructions\n- You will be provided a question, the system description, and relevant task context.\n- Interact with the Salesforce instance to build Salesforce Object Query Language (SOQL) or Salesforce Object Search Language (SOSL) queries as appropriate, to help you answer the question.\n- Salesforce Object Search Language (SOSL) can be used to construct text-based search queries against the search index.\n- Your generation should always be an Action command and NOTHING ELSE. Generate only one Action command.\n- DO NOT generate ANY system observation, you will receive this based on your Action command.\n- If no record is found matching the requirements mentioned, just return 'None'.\n- If the user's request is unclear or under-specified, use the respond action to ask for clarification before proceeding with queries or submitting an answer.\n\nHere is a description of h

In [41]:
# States
cap_results[0][1]

{'response': 1,
 'error': False,
 'done': True,
 'steps': 3,
 'executions': [{'query': "SELECT Id, Body__c, CreatedDate FROM VoiceCallTranscript__c WHERE LeadId__c = '00QWt0000089AekMAE' ORDER BY CreatedDate DESC LIMIT 1",
   'sql': "SELECT Id,Body__c,CreatedDate FROM VoiceCallTranscript__c WHERE LeadId__c = '00QWt0000089AekMAE' ORDER BY CreatedDate DESC LIMIT 1",
   'result': [{'Id': 'a05Wt000003SukEIAS',
     'Body__c': "[2023-10-21T10:02:00] Ava Sullivan: Hi Ali, this is Ava Sullivan from TechSolutions. How are you today?\n[2023-10-21T10:02:10] Ali Hussein: Hi Ava, I'm doing well, thank you. How about you?\n[2023-10-21T10:02:20] Ava Sullivan: I'm great, thank you for asking! I'm glad we could connect. I was looking over the details you provided about your interest in the PulseSim Pro. Is now a good time to discuss?\n[2023-10-21T10:02:30] Ali Hussein: Yes, now works for me. I'm interested to learn more and see how it fits with what we need.\n[2023-10-21T10:02:40] Ava Sullivan: Fantas

In [42]:
# Base Question
cap_results[0][2]


{'idx': 0,
 'answer': ['Authority'],
 'task': 'lead_qualification',
 'persona': 'You are quality-focused, maintaining high standards and attention to detail in all work.',
 'metadata': {'optional': '# Domain Details\n## Quarters of the Year\n- Q1: January 1 to March 31 (both inclusive).\n- Q2: April 1 to June 30 (both inclusive).\n- Q3: July 1 to September 30 (both inclusive).\n- Q4: October 1 to December 31 (both inclusive).\n## Seasons\n- Winter (2024): December 1, 2024 to February 28/29, 2025 (both inclusive), specifying the year to avoid ambiguity.\n- Spring: March 1 to May 31 (both inclusive).\n- Summer: June 1 to August 31 (both inclusive).\n- Autumn/Fall: September 1 to November 30 (both inclusive).\n## Time Periods\n- Past 2 quarters: This refers to any timeframe spanning two quarters back from a specified `end_date`. (i.e. begins on the date calculated by subtracting six months from the end_date and ends on the end_date) That translates to a six-month period retrospectively fr

# GRPO Full Task Text Data

In [30]:
dflq = pd.read_json('lead_qualification_train.jsonl', lines=True)
dfa = pd.read_json('activity_priority_train.jsonl', lines=True)
df = pd.concat([dflq, dfa]).reset_index()
df['output'] = [f'<execute> {q} </execute>' for q in df['output']]

In [26]:
temp_env = crm_agent.Environment()

In [21]:
def string_maker(gt_answer: list):
    if len(gt_answer) > 1:
        return ', '.join(gt_answer)
    else:
        return str(gt_answer[0])


In [22]:
lq_answers = [string_maker(d['answer']) for d in temp_env.dataset if d["task"] == 'lead_qualification']
ap_answers = [string_maker(d['answer']) for d in temp_env.dataset if d["task"] == 'activity_priority']

In [32]:
gt_query_results = []
for output in df['output']:
    query_command = crm_agent.message_action_parser(output)
    # query_command['query'] = soql_to_sql(query_command['content'])
    gt_query_results.append(temp_env.db_query(query_command))

In [34]:
df['query_results'] = gt_query_results
df['answers'] = lq_answers + ap_answers

In [36]:
df.to_csv('grpo_training_data.csv', index=False)

# Review

In [6]:
eval_df = pd.read_csv('SLM_Evaluation.csv')

In [21]:
# from crm_testing_utils import soql_to_sql
from sentence_transformers import SentenceTransformer, util

In [58]:
# q = {'name': 'execute', 'content': "SELECT Id, StageName FROM Opportunity WHERE Id = '006Wt000007BAMjIAO'"}
# q = {'name': 'execute', 'content': "SELECT Id FROM Task WHERE WhatId = '006Wt000007BAMjIAO' AND Status = 'Not Started'"}
q = {'name': 'execute', 'content': "SELECT Id FROM Task WHERE WhatId = '006Wt000007BAMjIAO' AND Status = 'Not Started' AND WhatId IN (SELECT Id FROM Opportunity WHERE StageName != 'Negotiation')"}
# q = {'name': 'execute', 'content': "SELECT Id FROM Task WHERE Id IN ('00TWt000002yqxYMAQ','00TWt000002yt7kMAA','00TWt000002z0oXMAQ','00TWt000002zAXcMAM','00TWt000002zC9fMAE','00TWt000002zD7MMAU','00TWt000002zDVXMA2','00TWt000002zEMgMAM','00TWt000002zEjMMAU','00TWt000002zEpkMAE','00TWt000002zFx9MAE','00TWt000002zGrWMAU','00TWt000002zJEIMA2','00TWt000002zKBxMAM') AND WhatId IN (SELECT Id FROM Opportunity WHERE StageName != 'Negotiation')"}

tq = temp_env.db_query(q)
tq

'Salesforce instance output: []'

In [54]:
actual_list = ['00TWt000002zEpkMAE', '00TWt000002zC9fMAE', '00TWt000002yt7kMAA', '00TWt000002zFx9MAE', '00TWt000002zEMgMAM', '00TWt000002zGrWMAU']
[a in tq for a in actual_list]

[True, True, True, True, True, True]

In [14]:
prediction_query_results = []
for pred in eval_df['predicted']:
    query_command = crm_agent.message_action_parser(pred)
    # query_command['query'] = soql_to_sql(query_command['content'])
    prediction_query_results.append(temp_env.db_query(query_command))


Query Error: no such column: Title
Query Error: near "FILTER": syntax error
Query Error: no such table: Need__c
Query Error: Incorrect number of bindings supplied. The current statement uses 1, and there are 0 supplied.
Query Error: near "FILTER": syntax error
Query Error: near "FILTER": syntax error
Query Error: near "RETURNING": syntax error
Query Error: near "FILTER": syntax error
Query Error: no such table: Lead__c
Query Error: no such column: StageId
Query Error: no such column: UserId
Query Error: no such column: OpportunityId
Query Error: near "FROM": syntax error
Query Error: no such column: StageId
Query Error: no such column: OpportunityId
Query Error: no such column: StageId
Query Error: near "POLL": syntax error
Query Error: near "Find": syntax error
Query Error: near "SOSL": syntax error
Query Error: near "SHOULD": syntax error
Query Error: near "POLL": syntax error
Query Error: near "FIND": syntax error
Query Error: near "FIND": syntax error
Query Error: no such column: N

In [19]:
actual_query_results = []
for actual in eval_df['actual']:
    if actual.startswith('<execute>'):
        actual_query = crm_agent.message_action_parser(actual)
    # query_command['query'] = soql_to_sql(query_command['content'])
    else:
        actual_query = {"name": "execute", "content": actual}
    actual_query_results.append(temp_env.db_query(actual_query))


In [22]:
model = SentenceTransformer('all-MiniLM-L6-v2')
def get_sentence_similarity(sent1, sent2):
    embeddings = model.encode([sent1, sent2])
    similarity_score = util.cos_sim(embeddings[0], embeddings[1])
    return similarity_score.item()


In [30]:
prediction_similarity_scores = [max(0,get_sentence_similarity(pred, actual)) for pred, actual in zip(prediction_query_results, actual_query_results)]

In [32]:
eval_df['prediction_query_results'] = prediction_query_results
eval_df['actual_query_results'] = actual_query_results
eval_df['prediction_similarity_scores'] = prediction_similarity_scores

In [34]:
eval_df.to_csv('slm_evaluated.csv', index=False)

In [24]:
all_csv_files = glob.glob(os.path.join('/Users/micksmith/Library/CloudStorage/GoogleDrive-csmith715@gmail.com/My Drive/Neuromatic/SLM-Training/', "*.csv"))

In [25]:
list_of_dfs = []
for filename in all_csv_files:
    model_name = filename.split('/')[-1]
    df = pd.read_csv(filename)
    df['model_name'] = model_name
    list_of_dfs.append(df)
combined_df = pd.concat(list_of_dfs, ignore_index=True)

In [27]:
combined_df.to_csv('full_slm_results.csv', index=False)

# GRPO Modeling

In [8]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

In [4]:
# base = AutoModelForCausalLM.from_pretrained(
#     "Qwen/Qwen3-4B-Instruct-2507",
#     # device_map="auto",
#     # torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float16,
#     # load_in_4bit=True,
# )
#
# ft = PeftModel.from_pretrained(base, '/Users/micksmith/Library/CloudStorage/GoogleDrive-csmith715@gmail.com/My Drive/Neuromatic/SLM-Training/qwen3-4b-grpo-lora-adapter')
# ft.eval()

In [35]:
def mystery(s):
    return "".join(sorted(set(s), reverse=True))

print(mystery("programming"))

rponmiga


In [9]:
BASE = "Qwen/Qwen3-4B-Instruct-2507"
ADAPTER = "/Users/micksmith/Library/CloudStorage/GoogleDrive-csmith715@gmail.com/My Drive/Neuromatic/SLM-Training/qwen3-4b-grpo-lora-adapter"

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
dtype = torch.float16 if device.type == "mps" else torch.float32

tokenizer = AutoTokenizer.from_pretrained(ADAPTER, use_fast=True)

base = AutoModelForCausalLM.from_pretrained(
    BASE,
    torch_dtype=dtype,
    low_cpu_mem_usage=True,
)

ft = PeftModel.from_pretrained(base, ADAPTER)
ft.eval()
ft = ft.to(device)
ft = ft.merge_and_unload().to(device)
ft.eval()


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2560)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=2560, out_features=4096, bias=False)
          (k_proj): Linear(in_features=2560, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2560, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=2560, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=2560, out_features=9728, bias=False)
          (up_proj): Linear(in_features=2560, out_features=9728, bias=False)
          (down_proj): Linear(in_features=9728, out_features=2560, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen3RMSNorm((2560,), eps=1e-06)
        (post_attention_layer

In [17]:

# def generate_qwen_response(message_list):
#     prompt = tokenizer.apply_chat_template(message_list, tokenize=False, add_generation_prompt=True)
#     inputs = tokenizer(prompt, return_tensors="pt").to(ft.device)
#
#     with torch.no_grad():
#         out = ft.generate(
#             **inputs,
#             max_new_tokens=32,
#             do_sample=False
#         )
#
#     result = tokenizer.decode(out[0], skip_special_tokens=True)
#     end_text = "LABELS must be None or a comma-separated subset of: Authority, Budget, Timeline, Need.\nassistant\n"
#     answer = result.partition(end_text)[2]
#
#     return answer

def generate_qwen_response(message_list: list, tokens: int = 8) -> str:
    prompt = tokenizer.apply_chat_template(message_list, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.inference_mode():
        out = ft.generate(
            **inputs,
            max_new_tokens=tokens,
            do_sample=False,
            use_cache=True,
        )

    # Only decode new tokens (avoid decoding the whole prompt)
    gen_tokens = out[0, inputs["input_ids"].shape[1]:]
    answer = tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()
    return answer


In [32]:
print('You are a lead qualification assistant.\nYou will be given call transcript excerpts (tool output).\nReturn exactly ONE Action command of the form:\n<respond> LABELS </respond>\nwhere LABELS is either:\n- None\n- or a comma-separated subset of: Authority, Budget, Timeline, Need\nNo other text.')

You are a lead qualification assistant.
You will be given call transcript excerpts (tool output).
Return exactly ONE Action command of the form:
<respond> LABELS </respond>
where LABELS is either:
- None
- or a comma-separated subset of: Authority, Budget, Timeline, Need
No other text.


In [1]:
sample = {'prompt': [{'content': 'You are a lead qualification assistant.\nYou will be given call transcript excerpts (tool output).\nReturn exactly ONE Action command of the form:\n<respond> LABELS </respond>\nwhere LABELS is either:\n- None\n- or a comma-separated subset of: Authority, Budget, Timeline, Need\nNo other text.',
   'role': 'system'},
  {'content': 'Review the latest transcript and determine whether this lead should be qualified.',
   'role': 'user'},
  {'content': "<execute> SELECT Id, Body__c, CreatedDate, LeadId__c FROM VoiceCallTranscript__c WHERE LeadId__c = '00QWtRGHOY32nfr5py' </execute>",
   'role': 'assistant'},
  {'content': 'Salesforce instance output: [{\'Id\': \'a05WtBTW5ZE9LFaez7\', \'Body__c\': "[2023-10-09T10:00:00] Amir Brown: Hi Sam, thanks for taking the call. How are things going?\\\\n[2023-10-09T10:00:25] Sam Garcia: Doing well—happy to chat.\\\\n[2023-10-09T10:00:50] Amir Brown: We\'re in financial services and evaluating a lead scoring platform, but it’s early.\\\\n[2023-10-09T10:00:19] Sam Garcia: We’re still exploring options; nothing decided.\\\\n[2023-10-09T10:00:41] Sam Garcia: I can share this with the team and get back to you.\\\\n[2023-10-09T10:02:10] Amir Brown: Great—I\'ll follow up with next steps and a quick recap.", \'CreatedDate\': \'2023-10-03T10:00:00.000+0000\', \'LeadId__c\': \'00QWtRGHOY32nfr5py\'}]',
   'role': 'tool'},
  {'content': 'Now provide the final qualification result.\nReturn exactly one action command:\n<respond> LABELS </respond>\nLABELS must be None or a comma-separated subset of: Authority, Budget, Timeline, Need.',
   'role': 'user'}],
 'ground_truth': '<respond> None </respond>'}

In [82]:
sample['prompt']

[{'content': 'You are a lead qualification assistant.\nYou will be given call transcript excerpts (tool output).\nReturn exactly ONE Action command of the form:\n<respond> LABELS </respond>\nwhere LABELS is either:\n- None\n- or a comma-separated subset of: Authority, Budget, Timeline, Need\nNo other text.',
  'role': 'system'},
 {'content': 'Review the latest transcript and determine whether this lead should be qualified.',
  'role': 'user'},
 {'content': "<execute> SELECT Id, Body__c, CreatedDate, LeadId__c FROM VoiceCallTranscript__c WHERE LeadId__c = '00QWtRGHOY32nfr5py' </execute>",
  'role': 'assistant'},
 {'content': 'Salesforce instance output: [{\'Id\': \'a05WtBTW5ZE9LFaez7\', \'Body__c\': "[2023-10-09T10:00:00] Amir Brown: Hi Sam, thanks for taking the call. How are things going?\\\\n[2023-10-09T10:00:25] Sam Garcia: Doing well—happy to chat.\\\\n[2023-10-09T10:00:50] Amir Brown: We\'re in financial services and evaluating a lead scoring platform, but it’s early.\\\\n[2023-10

In [29]:
qwen_final_answers = [generate_qwen_response(tm) for tm in test_messages]

RuntimeError: Invalid buffer size: 604.88 GiB

In [28]:
test_actual_answers

[['Authority'],
 ['Authority'],
 ['Timeline'],
 ['Authority'],
 ['Budget'],
 ['Authority'],
 ['Need'],
 [None],
 [None],
 ['Need']]